In [1]:
import pandas as pd
import openai
import time
import json
from serpapi import GoogleSearch
import pandas as pd

from src.utils import *
import warnings
warnings.filterwarnings('ignore')

c:\Users\RodrigoMartínezAlons\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df=pd.read_csv('results.csv')
df_company=pd.read_csv('data/company_names.csv')
df_brand=pd.read_csv('data/brand_names.csv')
df_alias=pd.read_csv('data/alias_names.csv')

In [3]:
with open("config.json", "r") as file:
    config = json.load(file)

serp_api_key = config["serp_api_key"]
openai.api_key = config["openai_api_key"]
google_api_key = config["google_custom_search"]["api_key"]
search_engine_id = config["google_custom_search"]["search_engine_id"]

In [4]:
df=df[['Original Name', 'Standardized Name']]

In [5]:
df.tail(20)

,Original Name,Standardized Name
1530,ERICO International Corp,ERICO
1531,Crescent Manufacturing Operating Co.,Crescent Manufacturing
1532,Plasti-Bond/Perma-Cote/Korkap,Plasti-Bond
1533,"Steel City Bolt & Screw, LLC",Steel City Bolt & Screw
1534,Legrand Electric Limited,Legrand
1535,"Haydon Corporation, Inc.",Haydon
1536,Anvil International,Anvil
1537,PASS & SEYMOUR,PASS & SEYMOUR
1538,"Pro Systems, LLC",Pro Systems
1539,Shawmut Corporation,Shawmut


### 1) Using Google Search API

In [6]:
df_test=df.iloc[0:10]

In [7]:
# Apply search function
df_test["Existence Check"] = df_test["Standardized Name"].apply(search_company)

In [ ]:
params = {"engine": "google", "q": f"{'CKD'} company","api_key": serp_api_key}
search = GoogleSearch(params)
results = search.get_dict()
results

{'search_metadata': {'id': '67ac6541aad5afc3325a0dd6',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/14374ae95d2ce293/67ac6541aad5afc3325a0dd6.json',
  'created_at': '2025-02-12 09:09:21 UTC',
  'processed_at': '2025-02-12 09:09:21 UTC',
  'google_url': 'https://www.google.com/search?q=CKD+company&oq=CKD+company&sourceid=chrome&ie=UTF-8',
  'raw_html_file': 'https://serpapi.com/searches/14374ae95d2ce293/67ac6541aad5afc3325a0dd6.html',
  'total_time_taken': 0.53},
 'search_parameters': {'engine': 'google',
  'q': 'CKD company',
  'google_domain': 'google.com',
  'device': 'desktop'},
 'search_information': {'query_displayed': 'CKD company',
  'total_results': 482000000,
  'time_taken_displayed': 0.34,
  'organic_results_state': 'Results for exact spelling'},
 'knowledge_graph': {'title': 'CKD Corporation',
  'type': 'Machinery manufacturing company',
  'entity_type': 'companies, chains, shopping_merchant',
  'kgmid': '/g/121n48lr',
  'knowledge_graph_search_lin

In [19]:
df_test.head(10)

,Original Name,Standardized Name,Existence Check
0,CKD,CKD,Exists
1,Mitsubishi Electric,Mitsubishi,Exists
2,Habasit,Habasit,Exists
3,Nordson,Nordson,Exists
4,CONVUM,CONVUM,Exists
5,HABASIT,HABASIT,Exists
6,KOGANEI,KOGANEI,Exists
7,Continental,Continental,Exists
8,KEYENCE,KEYENCE,Exists
9,Maruyasu,Maruyasu,Exists


#### 1.1) Google Custom Search API 

In [15]:
# Example: Search for CKD company
company_name = "CKD company"
title, link = google_search(company_name)
print(f"Company: {title}\nWebsite: {link}")

Company: CKD Pneumatic Components Control Components Automatic ...
Website: https://www.ckdusa.com/


In [38]:
df_try=df.iloc[0:100]

In [ ]:
# Apply the search function to all rows
df_try[["Exists", "Website"]] = df_try["Standardized Name"].apply(lambda name: pd.Series(google_search_df(name)))

In [40]:
df_try.head()

,Original Name,Standardized Name,Exists,Website
0,CKD,CKD,Yes,https://www.kidney.org/kidney-topics/chronic-k...
1,Mitsubishi Electric,Mitsubishi,Yes,https://www.mitsubishicars.com/
2,Habasit,Habasit,Yes,https://www.habasit.com/en-US
3,Nordson,Nordson,Yes,https://www.nordson.com/en
4,CONVUM,CONVUM,Yes,https://convum.co.jp/en/


In [41]:
df_try['Exists'].value_counts()

Exists
Yes    99
No      1
Name: count, dtype: int64

In [43]:
df_try[df_try['Exists']=='No']

,Original Name,Standardized Name,Exists,Website
92,yasugkawa,yasugkawa,No,


In [44]:
company_name = "yasugkawa"
title, link = google_search(company_name)
print(f"Company: {title}\nWebsite: {link}")

Company: No results found
Website: 


In [46]:
company_name = "yaskawa"
title, link = google_search(company_name)
print(f"Company: {title}\nWebsite: {link}")

Company: Yaskawa America Inc. Home - Yaskawa
Website: https://www.yaskawa.com/


### 2) Using Bing Search API

In [ ]:
from azure.cognitiveservices.search.websearch import WebSearchClient
from msrest.authentication import CognitiveServicesCredentials

BING_API_KEY = "your_bing_api_key"
client = WebSearchClient(endpoint="https://api.bing.microsoft.com/", credentials=CognitiveServicesCredentials(BING_API_KEY))

def search_bing(company_name):
    query = f"{company_name} official website"
    search_results = client.web.search(query=query)

    if search_results.web_pages:
        return "Exists"
    else:
        return "Not Found"

df["Existence_Check_Bing"] = df["Standardized Name"].apply(search_bing)

Could not create the Bing Search v7 in the Azure Portal

Error type
Your account is not approved for this resource. Contact BingAPIMS@microsoft.com with any questions.

 (Code: ApiSetDisabledForCreation)

### 3) Direct Web Scrapping

In [ ]:
import requests
from bs4 import BeautifulSoup

def google_search(company_name):
    headers = {"User-Agent": "Mozilla/5.0"}
    search_url = f"https://www.google.com/search?q={company_name}+company"
    
    response = requests.get(search_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Check if search results contain company websites
    if "No results found" not in soup.text:
        return "Exists"
    return "Not Found"

df_test["Existence_Check_BS"] = df_test["Standardized Name"].apply(google_search)

C:\Users\RodrigoMartínezAlons\AppData\Local\Temp\ipykernel_29524\817398160.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["Existence_Check_BS"] = df_test["Standardized Name"].apply(google_search)


In [28]:
headers = {"User-Agent": "Mozilla/5.0"}
search_url = f"https://www.google.com/search?q={'CKD'}+company"

response = requests.get(search_url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

In [29]:
soup

<!DOCTYPE html>
<html dir="ltr" lang="de"><head><style nonce="3GBUdyhIW7muLwEy2zhhiQ">
a, a:link, a:visited, a:active, a:hover {
  color: #1a73e8;
  text-decoration: none;
}
body {
  font-family: Roboto, Helvetica, Arial, sans-serif;
  text-align: center;
  -ms-text-size-adjust: 100%;
  -moz-text-size-adjust: 100%;
  -webkit-text-size-adjust: 100%;
}
.box {
  border: 1px solid #dadce0;
  box-sizing: border-box;
  border-radius: 8px;
  margin: 24px auto 5px auto;
  max-width: 800px;
  padding: 24px;
}
.youtubeContainerUIModernization,
.boxUIModernization {
  box-sizing: border-box;
  margin-left: auto;
  margin-right: auto;
  max-width: 800px;
}
.signInContainerUIModernization {
    display: flex;
    justify-content: flex-end;
}
h1 {
  color: #2c2c2c;
  font-size: 24px;
  hyphens: auto;
  margin: 24px 0;
}
.openInNewIcon,
.openInNewIconYouTube {
  height: 12px;
  width: 12px;
  margin-bottom: -2px;
  margin-left: 2px;

  
}
.openInNewIcon {
  fill: #0b57d0; // gm3-primary-blue
}
.openI

In [22]:
df_test.head()

,Original Name,Standardized Name,Existence Check,Existence_Check_BS
0,CKD,CKD,Exists,Exists
1,Mitsubishi Electric,Mitsubishi,Exists,Exists
2,Habasit,Habasit,Exists,Exists
3,Nordson,Nordson,Exists,Exists
4,CONVUM,CONVUM,Exists,Exists


### 4) GPT

### 5) Benchmarks

Benchmarking the standardization with the data in Pigeon

In [8]:
brand_count = df_brand.groupby("company_id").size().reset_index(name="brand_count")
df_companies_brands = df_company.merge(brand_count, on="company_id", how="left").fillna(0)
display(df_companies_brands)

,company_id,company_name,created_at,updated_at,company_status,is_mfr,is_supplier,brand_count
0,5201,2J Antenna,2023-08-25 12:48:41.39556,2023-08-25 12:48:41.39556,open,1.0,0.0,1.0
1,5212,A&S,2023-08-25 12:48:41.39556,2023-08-25 12:48:41.39556,open,1.0,0.0,1.0
2,5213,A. Schweiger,2023-08-25 12:48:41.39556,2023-08-25 12:48:41.39556,open,1.0,0.0,1.0
3,5214,A.C.Ryan,2023-08-25 12:48:41.39556,2023-08-25 12:48:41.39556,open,1.0,0.0,1.0
4,5215,A+P,2023-08-25 12:48:41.39556,2023-08-25 12:48:41.39556,open,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...
5374,10642,W.Diamant,2024-08-09 07:48:18.653927,2024-08-09 07:48:18.653927,0,0.0,0.0,1.0
5375,10748,W. Schmitz,2024-08-13 09:12:39.773154,2024-08-13 09:12:39.773154,0,0.0,0.0,1.0
5376,10843,PHOENIX Armaturenwerke,2024-08-28 08:54:04.923278,2024-08-28 08:54:04.923278,0,0.0,0.0,1.0
5377,10896,LÖSCH,2024-08-28 14:05:40.653313,2024-08-28 14:05:40.653313,0,0.0,0.0,1.0


In [9]:
df_merged = df_brand.merge(df_company, on="company_id", how="left")
df_final = df_alias.merge(df_merged, on="brand_id", how="left")

df_final = df_final.drop(columns=['created_at', 'updated_at', 'created_at_x', 'updated_at_x', 'created_at_y', 'updated_at_y'])

In [10]:
df_final


,alias_id,brand_id,alias_name,company_id,brand_name,company_name,company_status,is_mfr,is_supplier
0,7535,5466,1A Tapes E. K.,5200,1A TAPES E. K.1,1A Tapes E. K.,open,1.0,NaN
1,7537,5467,2J Antenna,5201,2J ANTENNA,2J Antenna,open,1.0,NaN
2,7538,5468,2 MANET,5202,2MANET,2MANET,open,1.0,NaN
3,7539,5469,3D Printz,5203,3D PRINTZ,3D Printz,open,1.0,NaN
4,7540,5470,3D Systems,5204,3D SYSTEMS,3D Systems,open,1.0,NaN
...,...,...,...,...,...,...,...,...,...
8910,17029,11848,BERNSTEIN TOOLS,5638,BERNSTEIN TOOLS,Bernstein,open,1.0,NaN
8911,17030,11849,MCMASTER,10990,MCMASTER,MCMASTER-CARR,NaN,NaN,NaN
8912,17031,11849,MCMA,10990,MCMASTER,MCMASTER-CARR,NaN,NaN,NaN
8913,17062,7606,Tianjin IMA Machinery Co.,7223,IMA,IMA,close,NaN,1.0


In [11]:
df_final.shape

(8915, 9)

In [12]:
df_repeated = df_final[df_final['company_name'].duplicated(keep=False)]

In [13]:
df_repeated = df_repeated.sort_values(by='company_name', ascending=True)
df_repeated.head(30)

,alias_id,brand_id,alias_name,company_id,brand_name,company_name,company_status,is_mfr,is_supplier
7024,14852,10825,A.R. WILFLEY & SONS,10330,A.R. WILFLEY & SONS,A.R. WILFLEY & SONS,NaN,NaN,NaN
7025,14853,10825,"A.R. WILFLEY & SONS, INC",10330,A.R. WILFLEY & SONS,A.R. WILFLEY & SONS,NaN,NaN,NaN
8674,16626,11545,AMK,10901,Aachener Misch- und Knetmaschinenfabrik Peter...,Aachener Misch- und Knetmaschinenfabrik Peter...,NaN,NaN,NaN
8673,16625,11545,Aachener Misch- und Knetmaschinenfabrik Peter...,10901,Aachener Misch- und Knetmaschinenfabrik Peter...,Aachener Misch- und Knetmaschinenfabrik Peter...,NaN,NaN,NaN
8672,16624,11545,Aachener Misch- und Knetmaschinenfabrik Peter...,10901,Aachener Misch- und Knetmaschinenfabrik Peter...,Aachener Misch- und Knetmaschinenfabrik Peter...,NaN,NaN,NaN
7934,15838,11255,Dr. Baus,10690,Dr. Baus,Dr. Baus,NaN,NaN,NaN
7933,15837,11255,Dr. Baus GmbH,10690,Dr. Baus,Dr. Baus,NaN,NaN,NaN
7004,14746,10745,KENNA,10247,KENNA,KENNA,NaN,NaN,NaN
7006,14748,10746,KENNAMETAL,10247,NaN,KENNA,NaN,NaN,NaN
7005,14747,10745,KENNA METAL,10247,KENNA,KENNA,NaN,NaN,NaN


### 5.1) For Alias

In [14]:
df_repeated.head()

,alias_id,brand_id,alias_name,company_id,brand_name,company_name,company_status,is_mfr,is_supplier
7024,14852,10825,A.R. WILFLEY & SONS,10330,A.R. WILFLEY & SONS,A.R. WILFLEY & SONS,NaN,NaN,NaN
7025,14853,10825,"A.R. WILFLEY & SONS, INC",10330,A.R. WILFLEY & SONS,A.R. WILFLEY & SONS,NaN,NaN,NaN
8674,16626,11545,AMK,10901,Aachener Misch- und Knetmaschinenfabrik Peter...,Aachener Misch- und Knetmaschinenfabrik Peter...,NaN,NaN,NaN
8673,16625,11545,Aachener Misch- und Knetmaschinenfabrik Peter...,10901,Aachener Misch- und Knetmaschinenfabrik Peter...,Aachener Misch- und Knetmaschinenfabrik Peter...,NaN,NaN,NaN
8672,16624,11545,Aachener Misch- und Knetmaschinenfabrik Peter...,10901,Aachener Misch- und Knetmaschinenfabrik Peter...,Aachener Misch- und Knetmaschinenfabrik Peter...,NaN,NaN,NaN


In [ ]:
# df_repeated_standardized_alias = classify_manufacturers_new(df_repeated["alias_name"].iloc[0:10])

In [ ]:
df_repeated_standardized_alias = standardize_manufacturers_df(df_repeated.iloc[0:10], input_col="alias_name", output_col="Standardized_Alias_Name")

In [18]:
df_repeated_standardized_alias

,alias_id,brand_id,alias_name,company_id,brand_name,company_name,company_status,is_mfr,is_supplier,Standardized_Alias_Name
7024,14852,10825,A.R. WILFLEY & SONS,10330,A.R. WILFLEY & SONS,A.R. WILFLEY & SONS,NaN,NaN,NaN,A.R. Wilfley
7025,14853,10825,"A.R. WILFLEY & SONS, INC",10330,A.R. WILFLEY & SONS,A.R. WILFLEY & SONS,NaN,NaN,NaN,A.R. Wilfley & Sons
8674,16626,11545,AMK,10901,Aachener Misch- und Knetmaschinenfabrik Peter...,Aachener Misch- und Knetmaschinenfabrik Peter...,NaN,NaN,NaN,AMK
8673,16625,11545,Aachener Misch- und Knetmaschinenfabrik Peter...,10901,Aachener Misch- und Knetmaschinenfabrik Peter...,Aachener Misch- und Knetmaschinenfabrik Peter...,NaN,NaN,NaN,Aachener Misch- und Knetmaschinenfabrik Peter ...
8672,16624,11545,Aachener Misch- und Knetmaschinenfabrik Peter...,10901,Aachener Misch- und Knetmaschinenfabrik Peter...,Aachener Misch- und Knetmaschinenfabrik Peter...,NaN,NaN,NaN,Aachener Misch- und Knetmaschinenfabrik Peter ...
7934,15838,11255,Dr. Baus,10690,Dr. Baus,Dr. Baus,NaN,NaN,NaN,Dr. Baus
7933,15837,11255,Dr. Baus GmbH,10690,Dr. Baus,Dr. Baus,NaN,NaN,NaN,Dr. Baus
7004,14746,10745,KENNA,10247,KENNA,KENNA,NaN,NaN,NaN,KENNA
7006,14748,10746,KENNAMETAL,10247,NaN,KENNA,NaN,NaN,NaN,Kennametal
7005,14747,10745,KENNA METAL,10247,KENNA,KENNA,NaN,NaN,NaN,KENNA METAL


In [19]:
# df_repeated_standardized_alias[["Brand_Exists", "Brand_Website"]] = df_repeated_standardized_alias["Standardized_Brand_Name"].apply(lambda name: pd.Series(google_search_df(name)))
df_repeated_standardized_alias[["Alias_Exists", "Alias_Website"]] = df_repeated_standardized_alias["Standardized_Alias_Name"].apply(lambda name: pd.Series(google_search_df(name)))

In [20]:
df_repeated_standardized_alias.head()

,alias_id,brand_id,alias_name,company_id,brand_name,company_name,company_status,is_mfr,is_supplier,Standardized_Alias_Name,Alias_Exists,Alias_Website
7024,14852,10825,A.R. WILFLEY & SONS,10330,A.R. WILFLEY & SONS,A.R. WILFLEY & SONS,NaN,NaN,NaN,A.R. Wilfley,Yes,https://www.wilfley.com/
7025,14853,10825,"A.R. WILFLEY & SONS, INC",10330,A.R. WILFLEY & SONS,A.R. WILFLEY & SONS,NaN,NaN,NaN,A.R. Wilfley & Sons,Yes,https://www.wilfley.com/
8674,16626,11545,AMK,10901,Aachener Misch- und Knetmaschinenfabrik Peter...,Aachener Misch- und Knetmaschinenfabrik Peter...,NaN,NaN,NaN,AMK,Yes,https://www.amkproducts.com/
8673,16625,11545,Aachener Misch- und Knetmaschinenfabrik Peter...,10901,Aachener Misch- und Knetmaschinenfabrik Peter...,Aachener Misch- und Knetmaschinenfabrik Peter...,NaN,NaN,NaN,Aachener Misch- und Knetmaschinenfabrik Peter ...,Yes,https://www.amk-mixer.com/
8672,16624,11545,Aachener Misch- und Knetmaschinenfabrik Peter...,10901,Aachener Misch- und Knetmaschinenfabrik Peter...,Aachener Misch- und Knetmaschinenfabrik Peter...,NaN,NaN,NaN,Aachener Misch- und Knetmaschinenfabrik Peter ...,Yes,https://www.amk-mixer.com/


Similarity, for having a benchmark in this case

In [21]:
df_repeated_standardized_alias["Similarity_Score"] = df_repeated_standardized_alias.apply(lambda row: hybrid_similarity(row['company_name'], row['Standardized_Alias_Name']), axis=1)

In [22]:
df_repeated_standardized_alias.head()

,alias_id,brand_id,alias_name,company_id,brand_name,company_name,company_status,is_mfr,is_supplier,Standardized_Alias_Name,Alias_Exists,Alias_Website,Similarity_Score
7024,14852,10825,A.R. WILFLEY & SONS,10330,A.R. WILFLEY & SONS,A.R. WILFLEY & SONS,NaN,NaN,NaN,A.R. Wilfley,Yes,https://www.wilfley.com/,0.786
7025,14853,10825,"A.R. WILFLEY & SONS, INC",10330,A.R. WILFLEY & SONS,A.R. WILFLEY & SONS,NaN,NaN,NaN,A.R. Wilfley & Sons,Yes,https://www.wilfley.com/,1.000
8674,16626,11545,AMK,10901,Aachener Misch- und Knetmaschinenfabrik Peter...,Aachener Misch- und Knetmaschinenfabrik Peter...,NaN,NaN,NaN,AMK,Yes,https://www.amkproducts.com/,0.066
8673,16625,11545,Aachener Misch- und Knetmaschinenfabrik Peter...,10901,Aachener Misch- und Knetmaschinenfabrik Peter...,Aachener Misch- und Knetmaschinenfabrik Peter...,NaN,NaN,NaN,Aachener Misch- und Knetmaschinenfabrik Peter ...,Yes,https://www.amk-mixer.com/,1.000
8672,16624,11545,Aachener Misch- und Knetmaschinenfabrik Peter...,10901,Aachener Misch- und Knetmaschinenfabrik Peter...,Aachener Misch- und Knetmaschinenfabrik Peter...,NaN,NaN,NaN,Aachener Misch- und Knetmaschinenfabrik Peter ...,Yes,https://www.amk-mixer.com/,1.000


The ones without perfect match

In [26]:
(df_repeated_standardized_alias[df_repeated_standardized_alias["Similarity_Score"] != 1])

,alias_id,brand_id,alias_name,company_id,brand_name,company_name,company_status,is_mfr,is_supplier,Standardized_Alias_Name,Alias_Exists,Alias_Website,Similarity_Score
7024,14852,10825,A.R. WILFLEY & SONS,10330,A.R. WILFLEY & SONS,A.R. WILFLEY & SONS,NaN,NaN,NaN,A.R. Wilfley,Yes,https://www.wilfley.com/,0.786
8674,16626,11545,AMK,10901,Aachener Misch- und Knetmaschinenfabrik Peter...,Aachener Misch- und Knetmaschinenfabrik Peter...,NaN,NaN,NaN,AMK,Yes,https://www.amkproducts.com/,0.066
7006,14748,10746,KENNAMETAL,10247,NaN,KENNA,NaN,NaN,NaN,Kennametal,Yes,https://www.kennametal.com/us/en/home.html,0.402
7005,14747,10745,KENNA METAL,10247,KENNA,KENNA,NaN,NaN,NaN,KENNA METAL,Yes,https://www.kennametal.com/us/en/home.html,0.572


Perfect matches

In [27]:
(df_repeated_standardized_alias[df_repeated_standardized_alias["Similarity_Score"] == 1])

,alias_id,brand_id,alias_name,company_id,brand_name,company_name,company_status,is_mfr,is_supplier,Standardized_Alias_Name,Alias_Exists,Alias_Website,Similarity_Score
7025,14853,10825,"A.R. WILFLEY & SONS, INC",10330,A.R. WILFLEY & SONS,A.R. WILFLEY & SONS,NaN,NaN,NaN,A.R. Wilfley & Sons,Yes,https://www.wilfley.com/,1.0
8673,16625,11545,Aachener Misch- und Knetmaschinenfabrik Peter...,10901,Aachener Misch- und Knetmaschinenfabrik Peter...,Aachener Misch- und Knetmaschinenfabrik Peter...,NaN,NaN,NaN,Aachener Misch- und Knetmaschinenfabrik Peter ...,Yes,https://www.amk-mixer.com/,1.0
8672,16624,11545,Aachener Misch- und Knetmaschinenfabrik Peter...,10901,Aachener Misch- und Knetmaschinenfabrik Peter...,Aachener Misch- und Knetmaschinenfabrik Peter...,NaN,NaN,NaN,Aachener Misch- und Knetmaschinenfabrik Peter ...,Yes,https://www.amk-mixer.com/,1.0
7934,15838,11255,Dr. Baus,10690,Dr. Baus,Dr. Baus,NaN,NaN,NaN,Dr. Baus,Yes,https://www.bradlybausdentistry.com/,1.0
7933,15837,11255,Dr. Baus GmbH,10690,Dr. Baus,Dr. Baus,NaN,NaN,NaN,Dr. Baus,Yes,https://www.bradlybausdentistry.com/,1.0
7004,14746,10745,KENNA,10247,KENNA,KENNA,NaN,NaN,NaN,KENNA,Yes,https://www.cisco.com/c/en/us/products/securit...,1.0


In [ ]:
# df_repeated_standardized_alias.to_excel('df_standard_non_perfect.xlsx')

### 5.2) For Brands

In [32]:
df_repeated_standardized_brand = standardize_manufacturers_df(df_repeated.iloc[0:10], "brand_name", 'Standardized_Brand_Name')

In [33]:
df_repeated_standardized_brand[["Brand_Exists", "Brand_Website"]] = df_repeated_standardized_brand["Standardized_Brand_Name"].apply(lambda name: pd.Series(google_search_df(name)))

In [34]:
df_repeated_standardized_brand["Similarity_Score"] = df_repeated_standardized_brand.apply(lambda row: hybrid_similarity(row['company_name'], row['Standardized_Brand_Name']), axis=1)

In [35]:
(df_repeated_standardized_brand[df_repeated_standardized_brand["Similarity_Score"] != 1]).shape

(2, 14)

In [36]:
# (df_repeated_standardized_brand[df_repeated_standardized_brand["Similarity_Score"] == 1])

In [37]:
(df_repeated_standardized_brand[df_repeated_standardized_brand["Similarity_Score"] != 1])

,alias_id,brand_id,alias_name,company_id,brand_name,company_name,company_status,is_mfr,is_supplier,Standardized_Alias_Name,Standardized_Brand_Name,Brand_Exists,Brand_Website,Similarity_Score
7025,14853,10825,"A.R. WILFLEY & SONS, INC",10330,A.R. WILFLEY & SONS,A.R. WILFLEY & SONS,NaN,NaN,NaN,A.R. Wilfley & Sons,A.R. Wilfley,Yes,https://www.wilfley.com/,0.786
7006,14748,10746,KENNAMETAL,10247,NaN,KENNA,NaN,NaN,NaN,KENNAMETAL,nan,Yes,https://nationalactionnetwork.net/,0.300
